In [125]:
import pandas as pd
import numpy as np 

dataFile="/Users/Prince.Bharti/Desktop/csv/ml-100k/u.data"
data=pd.read_csv(dataFile,sep="\t",header=None,names=['userId','itemId','rating','timestamp'])

userInfoFile="/Users/Prince.Bharti/Desktop/csv/ml-100k/u.user"
userInfo=pd.read_csv(userInfoFile,sep="|",header=None,index_col=False,names=['userId','age','gender','occupation','zip code'])

movieInfoFile="/Users/Prince.Bharti/Desktop/csv/ml-100k/u.item"
movieInfo=pd.read_csv(movieInfoFile,header=None,sep="|",index_col=False,names=['itemId','title'],usecols=[0,1],encoding = "ISO-8859-1") #encoding has been used because it was giving some kind of parsing error via stack overflow

#basically we will use only data and movie info by merging(~ sql join) both dataframes.
data=pd.merge(data,movieInfo,left_on='itemId',right_on='itemId')



### method to calculate top n movies rated  by a  user .

In [126]:
def topNMovies(n,activeUser):
    movies=data[data.userId==activeUser].sort_values(by=['rating'],ascending=[0]).title[0:n]
    return list(movies)
    



### neighbourhood based collaborative filtering approach for recommending movies to a particular user

#### For a neighbourhood based collaborative approach basically we deal with two methods/functions.
1.finding k nearest neighbours which are similar to the active users.

2.Prediced rating matrix with the help of k nearest neighbours.

In [131]:

import numpy as np
import pandas as pd


dataFile="/Users/Prince.Bharti/Desktop/csv/ml-100k/u.data"
data=pd.read_csv(dataFile,sep="\t",header=None,names=['userId','itemId','rating','timestamp'])
userInfoFile="/Users/Prince.Bharti/Desktop/csv/ml-100k/u.user"
userInfo=pd.read_csv(userInfoFile,sep="|",header=None,index_col=False,names=['userId','age','gender','occupation','zip code'])

movieInfoFile="/Users/Prince.Bharti/Desktop/csv/ml-100k/u.item"
movieInfo=pd.read_csv(movieInfoFile,header=None,sep="|",index_col=False,names=['itemId','title'],usecols=[0,1],encoding = "ISO-8859-1") #encoding has been used because it was giving some kind of parsing error via stack overflow

#basically we will use only data and movie info by merging(~ sql join) both dataframes.
data=pd.merge(data,movieInfo,left_on='itemId',right_on='itemId')
userItemRatingMatrix=pd.pivot_table(data,index='userId',values='rating',columns='itemId')



from scipy import spatial
def similarity(userItemRatingMatrix,user1, user2):
    commonItemList=[i for i in range(0,len(userItemRatingMatrix)) if user1[i]>0 and user2[i]>0]
    user1=np.array(user1[commonItemList])
    user2=np.array(user2[commonItemList])
    if(len(commonItemList)==0):
        return 0
    else:
        return  spatial.distance.correlation(user1,user2)

def nearestNeighbours(userItemRatingMatrix,activeUser,k):
#     for storing id and similarity value of k neighbours
    similarMatrix=pd.DataFrame(index=userItemRatingMatrix.index,columns=['similarity'])
    
#   iterating for each user in the userItemMatrix and finding the similarity with the 
#   active user and then storing in the similarMatrix
    for i in userItemRatingMatrix.index:
        user1=np.array(userItemRatingMatrix.loc[i])
        user2=np.array(userItemRatingMatrix.loc[activeUser])
        similarMatrix.loc[i,'similarity']=similarity(userItemRatingMatrix,user1,user2)
        similarMatrix=similarMatrix.sort_values(by=['similarity'],ascending=True)
    return similarMatrix[1:5]
    
    





In [138]:
nearestNeighboursData=nearestNeighbours(userItemRatingMatrix,2,5)
userItemRatingMatrix[nearestNeighbours.in]

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/scipy/spatial/distance.py:364: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(um, vm) / (norm(um) * norm(vm))


,similarity
userId,
607,-2.22045e-16
426,-2.22045e-16
237,0
571,0
